<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch03/ch03-01-numerical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

ON_COLAB: True
USE_GIT: True
Cloning into 'kagglebook'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 95 (delta 19), reused 87 (delta 18), pack-reused 1 (from 1)
Receiving objects: 100% (95/95), 3.13 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (19/19), done.


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [4]:
# 説明用に学習データとテストデータの元の状態を保存しておく
train_x_saved = train_x.copy()
test_x_saved = test_x.copy()

In [5]:
# 学習データとテストデータを返す関数
def load_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()
    return train_x, test_x

In [6]:
# 変換する数値変数をリストに格納
num_cols = ['age', 'height', 'weight', 'amount',
            'medical_info_a1', 'medical_info_a2', 'medical_info_a3', 'medical_info_b1']

In [7]:
# -----------------------------------
# 標準化
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import StandardScaler

In [8]:
# 学習データに基づいて複数列の標準化を定義
scaler = StandardScaler()
scaler.fit(train_x[num_cols])

StandardScaler()

In [9]:
# 変換後のデータで各列を置換
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

In [10]:
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import StandardScaler

In [11]:
# 学習データとテストデータを結合したものに基づいて複数列の標準化を定義
scaler = StandardScaler()
scaler.fit(pd.concat([train_x[num_cols], test_x[num_cols]]))

StandardScaler()

In [12]:
# 変換後のデータで各列を置換
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

In [13]:
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import StandardScaler

In [14]:
# 学習データとテストデータを別々に標準化（悪い例）
scaler_train = StandardScaler()
scaler_train.fit(train_x[num_cols])
train_x[num_cols] = scaler_train.transform(train_x[num_cols])
scaler_test = StandardScaler()
scaler_test.fit(test_x[num_cols])
test_x[num_cols] = scaler_test.transform(test_x[num_cols])

In [15]:
# -----------------------------------
# Min-Maxスケーリング
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import MinMaxScaler

In [16]:
# 学習データに基づいて複数列のMin-Maxスケーリングを定義
scaler = MinMaxScaler()
scaler.fit(train_x[num_cols])

MinMaxScaler()

In [17]:
# 変換後のデータで各列を置換
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

In [18]:
# -----------------------------------
# 対数変換
# -----------------------------------
x = np.array([1.0, 10.0, 100.0, 1000.0, 10000.0])

In [19]:
# 単に対数をとる
x1 = np.log(x)

In [20]:
# 1を加えたあとに対数をとる
x2 = np.log1p(x)

In [21]:
# 絶対値の対数をとってから元の符号を付加する
x3 = np.sign(x) * np.log(np.abs(x))

In [22]:
# -----------------------------------
# Box-Cox変換
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------

In [23]:
# 正の値のみをとる変数を変換対象としてリストに格納する
# なお、欠損値も含める場合は、(~(train_x[c] <= 0.0)).all() などとする必要があるので注意
pos_cols = [c for c in num_cols if (train_x[c] > 0.0).all() and (test_x[c] > 0.0).all()]

In [24]:
from sklearn.preprocessing import PowerTransformer

In [25]:
# 学習データに基づいて複数列のBox-Cox変換を定義
pt = PowerTransformer(method='box-cox')
pt.fit(train_x[pos_cols])

PowerTransformer(method='box-cox')

In [26]:
# 変換後のデータで各列を置換
train_x[pos_cols] = pt.transform(train_x[pos_cols])
test_x[pos_cols] = pt.transform(test_x[pos_cols])

In [27]:
# -----------------------------------
# Yeo-Johnson変換
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------

In [28]:
from sklearn.preprocessing import PowerTransformer

In [29]:
# 学習データに基づいて複数列のYeo-Johnson変換を定義
pt = PowerTransformer(method='yeo-johnson')
pt.fit(train_x[num_cols])

PowerTransformer()

In [30]:
# 変換後のデータで各列を置換
train_x[num_cols] = pt.transform(train_x[num_cols])
test_x[num_cols] = pt.transform(test_x[num_cols])

In [31]:
# -----------------------------------
# clipping
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
# 列ごとに学習データの1％点、99％点を計算
p01 = train_x[num_cols].quantile(0.01)
p99 = train_x[num_cols].quantile(0.99)

In [32]:
# 1％点以下の値は1％点に、99％点以上の値は99％点にclippingする
train_x[num_cols] = train_x[num_cols].clip(p01, p99, axis=1)
test_x[num_cols] = test_x[num_cols].clip(p01, p99, axis=1)

In [33]:
# -----------------------------------
# binning
# -----------------------------------
x = [1, 7, 5, 4, 6, 3]

pandasのcut関数でbinningを行う

In [34]:
# binの数を指定する場合
binned = pd.cut(x, 3, labels=False)
print(binned)
# [0 2 1 1 2 0] - 変換された値は3つのbinのどれに入ったかを表す

[0 2 1 1 2 0]


In [35]:
# binの範囲を指定する場合（3.0以下、3.0より大きく5.0以下、5.0より大きい）
bin_edges = [-float('inf'), 3.0, 5.0, float('inf')]
binned = pd.cut(x, bin_edges, labels=False)
print(binned)
# [0 2 1 1 2 0] - 変換された値は3つのbinのどれに入ったかを表す

[0 2 1 1 2 0]


In [36]:
# -----------------------------------
# 順位への変換
# -----------------------------------
x = [10, 20, 30, 0, 40, 40]

In [37]:
# pandasのrank関数で順位に変換する
rank = pd.Series(x).rank()
print(rank.values)
# はじまりが1、同順位があった場合は平均の順位となる
# [2. 3. 4. 1. 5.5 5.5]

[2.  3.  4.  1.  5.5 5.5]


In [38]:
# numpyのargsort関数を2回適用する方法で順位に変換する
order = np.argsort(x)
rank = np.argsort(order)
print(rank)
# はじまりが0、同順位があった場合はどちらかが上位となる
# [1 2 3 0 4 5]

[1 2 3 0 4 5]


In [39]:
# -----------------------------------
# RankGauss
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import QuantileTransformer

In [40]:
# 学習データに基づいて複数列のRankGaussによる変換を定義
transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution='normal')
transformer.fit(train_x[num_cols])

QuantileTransformer(n_quantiles=100, output_distribution='normal',
                    random_state=0)

In [41]:
# 変換後のデータで各列を置換
train_x[num_cols] = transformer.transform(train_x[num_cols])
test_x[num_cols] = transformer.transform(test_x[num_cols])